In [1]:
import random
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException
from bs4 import BeautifulSoup
import re
from concurrent.futures import ThreadPoolExecutor
import queue

USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.4 Safari/605.1.15",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:125.0) Gecko/20100101 Firefox/125.0",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36 OPR/110.0.0.0",
    "Mozilla/5.0 (iPad; CPU OS 13_5 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/83.0.4103.88 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (Linux; Android 10; SM-G973F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Mobile Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Edge/125.0.0.0",
]

PROXIES = [
    "156.228.92.31:3129",
    "156.228.0.13:3129",
    "156.233.93.173:3129",
    "154.94.13.160:3129",
    "156.228.88.226:3129",
    "156.228.104.95:3129",
    "156.233.92.165:3129",
    "45.202.76.228:3129",
    "156.228.79.52:3129",
    "156.253.170.228:3129",
    "154.213.194.185:3129",
    "156.228.118.158:3129",
    "156.253.169.158:3129",
    "154.213.197.86:3129",
    "45.201.11.124:3129",
    "156.233.88.78:3129",
    "156.228.113.222:3129",
    "156.240.99.247:3129",
    "156.228.78.245:3129",
    "156.228.86.169:3129",
    "156.228.109.114:3129",
    "156.228.117.121:3129",
    "156.228.190.211:3129",
    "154.213.199.205:3129",
    "154.91.171.73:3129",
    "156.253.166.33:3129",
    "156.228.97.228:3129",
    "154.213.197.130:3129",
    "156.242.44.13:3129",
    "156.228.182.63:3129",
    "156.228.80.149:3129",
    "45.201.11.164:3129",
    "154.213.196.210:3129",
    "156.233.73.141:3129",
    "156.228.190.26:3129",
    "156.253.173.49:3129",
    "156.228.108.247:3129",
    "156.228.177.83:3129",
    "156.228.124.109:3129",
    "156.253.177.130:3129",
    "154.213.166.20:3129",
    "156.248.86.30:3129",
    "156.253.174.15:3129",
    "156.228.82.203:3129",
    "156.253.171.166:3129",
    "45.202.78.33:3129",
    "156.249.58.62:3129",
    "156.233.95.155:3129",
    "156.228.78.42:3129",
    "156.253.172.127:3129",
    "45.202.77.155:3129",
    "156.233.72.246:3129",
    "154.213.196.40:3129",
    "156.253.172.180:3129",
    "156.228.98.161:3129",
    "156.242.37.109:3129",
    "156.253.168.108:3129",
    "156.253.172.238:3129",
    "156.228.178.163:3129",
    "156.253.173.187:3129",
    "156.228.90.170:3129",
    "154.213.161.141:3129",
    "154.94.12.115:3129",
    "156.253.171.107:3129",
    "156.248.85.158:3129",
    "156.228.174.35:3129",
    "154.94.13.213:3129",
    "154.213.164.98:3129",
    "156.253.165.189:3129",
    "156.228.99.98:3129",
    "156.233.74.54:3129",
    "154.213.167.201:3129",
    "156.228.88.122:3129",
    "156.242.38.242:3129",
    "154.213.204.37:3129",
    "45.202.77.81:3129",
    "156.242.37.187:3129",
    "156.228.87.233:3129",
    "156.228.118.83:3129",
    "154.214.1.219:3129",
    "156.228.113.182:3129",
    "156.228.118.106:3129",
    "156.249.61.253:3129",
    "156.228.102.186:3129",
    "156.228.175.5:3129",
    "156.228.77.183:3129",
    "154.213.166.14:3129",
    "156.242.35.229:3129",
    "156.249.62.49:3129",
    "156.248.86.254:3129",
    "156.228.88.149:3129",
    "156.228.92.28:3129",
    "156.242.39.184:3129",
    "156.228.82.53:3129",
    "156.242.35.200:3129",
    "156.249.137.107:3129",
    "154.213.198.63:3129",
    "156.228.112.86:3129",
    "45.201.11.84:3129",
    "156.228.114.78:3129"
]

In [2]:
def click_accept_continue(driver, timeout=2):
    try:
        btn = WebDriverWait(driver, timeout).until(
            EC.element_to_be_clickable(
                (By.XPATH, "//button[contains(@title,'Accept') or contains(text(),'Accept')]"))
        )
        if btn.is_displayed():
            btn.click()
            return True
    except Exception:
        pass
    # Cek iframe
    iframes = driver.find_elements(By.TAG_NAME, "iframe")
    for iframe in iframes:
        try:
            driver.switch_to.frame(iframe)
            btn = driver.find_element(By.XPATH, "//button[contains(@title,'Accept') or contains(text(),'Accept')]")
            btn.click()
            driver.switch_to.default_content()
            return True
        except Exception:
            driver.switch_to.default_content()
    driver.switch_to.default_content()
    return False

In [3]:
import random
import requests
from bs4 import BeautifulSoup

BASE_URL = "https://www.transfermarkt.com"

def get_random_user_agent():
    return random.choice(USER_AGENTS)

def get_random_proxy():
    return random.choice(PROXIES)

def extract_club_urls_from_league(league_url, timeout=30):
    headers = {"User-Agent": get_random_user_agent()}
    proxy = {"http": "http://" + get_random_proxy(), "https": "http://" + get_random_proxy()}
    try:
        print(f"[INFO] Loading: {league_url}")
        resp = requests.get(league_url, headers=headers, proxies=proxy, timeout=timeout)
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "html.parser")
        club_urls = []
        table = soup.select_one("table.items")
        for a in table.select("td.hauptlink a[href*='/startseite/verein/']"):
            partial_url = a.get("href")
            if partial_url:
                full_url = BASE_URL + partial_url.split("?")[0]
                if full_url not in club_urls:
                    club_urls.append(full_url)
        print(f"[INFO] Dapat {len(club_urls)} klub")
        return club_urls
    except Exception as e:
        print(f"[ERROR] {e}")
        return []

In [4]:
if __name__ == "__main__":
    # Liga Spanyol (La Liga)
    laliga_url = "https://www.transfermarkt.co.id/laliga/startseite/wettbewerb/ES1/saison_id/2024"
    clubs_laliga = extract_club_urls_from_league(laliga_url)
    print("[La Liga]")
    for url in clubs_laliga:
        print(url)
    print("\n")

    # Serie A Italia
    seriea_url = "https://www.transfermarkt.co.id/serie-a/startseite/wettbewerb/IT1/saison_id/2024"
    clubs_seriea = extract_club_urls_from_league(seriea_url)
    print("[Serie A]")
    for url in clubs_seriea:
        print(url)
    print("\n")

    # Bundesliga Jerman
    bundesliga_url = "https://www.transfermarkt.com/bundesliga/startseite/wettbewerb/L1/saison_id/2024"
    clubs_bundesliga = extract_club_urls_from_league(bundesliga_url)
    print("[Bundesliga]")
    for url in clubs_bundesliga:
        print(url)
    print("\n")

    # Premier League Inggris
    epl_url = "https://www.transfermarkt.com/premier-league/startseite/wettbewerb/GB1/saison_id/2024"
    clubs_epl = extract_club_urls_from_league(epl_url)
    print("[Premier League]")
    for url in clubs_epl:
        print(url)
    print("\n")

    # Ligue 1 Prancis
    ligue1_url = "https://www.transfermarkt.com/ligue-1/startseite/wettbewerb/FR1/saison_id/2024"
    clubs_ligue1 = extract_club_urls_from_league(ligue1_url)
    print("[Ligue 1]")
    for url in clubs_ligue1:
        print(url)
    print("\n")

[INFO] Loading: https://www.transfermarkt.co.id/laliga/startseite/wettbewerb/ES1/saison_id/2024
[INFO] Dapat 20 klub
[La Liga]
https://www.transfermarkt.com/real-madrid/startseite/verein/418/saison_id/2024
https://www.transfermarkt.com/fc-barcelona/startseite/verein/131/saison_id/2024
https://www.transfermarkt.com/atletico-madrid/startseite/verein/13/saison_id/2024
https://www.transfermarkt.com/athletic-bilbao/startseite/verein/621/saison_id/2024
https://www.transfermarkt.com/real-sociedad-san-sebastian/startseite/verein/681/saison_id/2024
https://www.transfermarkt.com/fc-villarreal/startseite/verein/1050/saison_id/2024
https://www.transfermarkt.com/real-betis-sevilla/startseite/verein/150/saison_id/2024
https://www.transfermarkt.com/fc-valencia/startseite/verein/1049/saison_id/2024
https://www.transfermarkt.com/fc-sevilla/startseite/verein/368/saison_id/2024
https://www.transfermarkt.com/fc-girona/startseite/verein/12321/saison_id/2024
https://www.transfermarkt.com/espanyol-barcelona/

In [5]:
def start_driver():
    # Randomize User-Agent and Proxy
    user_agent = get_random_user_agent()
    proxy = get_random_proxy()
    chrome_options = Options()
    chrome_options.add_argument(f"user-agent={user_agent}")
    chrome_options.add_argument("--headless=new")
    chrome_options.add_argument("--window-size=1920,1080")
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--log-level=3")
    chrome_options.add_argument("--disable-infobars")
    chrome_options.add_argument(f'--proxy-server=http://{proxy}')
    driver = webdriver.Chrome(options=chrome_options)
    return driver

In [6]:
def extract_player_urls_from_club(club_url, timeout=20):
    orig_url = club_url
    # Ubah ke /kader/
    if "/startseite/" in club_url:
        club_url = club_url.replace("/startseite/", "/kader/")
    headers = {"User-Agent": get_random_user_agent()}
    proxy_addr = get_random_proxy()
    proxies = {"http": "http://" + proxy_addr, "https": "http://" + proxy_addr}
    try:
        print(f"[INFO] Klub: {orig_url}\n      Proxy: {proxy_addr}\n      GET: {club_url}")
        resp = requests.get(club_url, headers=headers, proxies=proxies, timeout=timeout)
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "html.parser")
        player_urls = []
        for a in soup.select("table.items td.hauptlink a[href*='/profil/spieler/']"):
            partial_url = a.get("href")
            if partial_url:
                full_url = BASE_URL + partial_url.split("?")[0]
                if full_url not in player_urls:
                    player_urls.append(full_url)
        print(f"   [SUCCESS] {len(player_urls)} pemain ditemukan.")
        return player_urls
    except Exception as e:
        print(f"   [ERROR] Gagal scrape {orig_url} | Proxy: {proxy_addr} | {e}")
        return []

def extract_all_player_urls_for_league_parallel_nested(club_urls, max_workers=10, liga_nama=""):
    print(f"\n[LEAGUE INFO] {liga_nama}: {len(club_urls)} klub | Paralel: {max_workers}")
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        result = list(executor.map(extract_player_urls_from_club, club_urls))
    total_pemain = sum(len(x) for x in result)
    print(f"[LEAGUE DONE] {liga_nama}: {len(result)} klub, {total_pemain} pemain total.\n")
    return result

max_workers = 40

players_laliga     = extract_all_player_urls_for_league_parallel_nested(clubs_laliga, max_workers, "La Liga")
players_seriea     = extract_all_player_urls_for_league_parallel_nested(clubs_seriea, max_workers, "Serie A")
players_bundesliga = extract_all_player_urls_for_league_parallel_nested(clubs_bundesliga, max_workers, "Bundesliga")
players_epl        = extract_all_player_urls_for_league_parallel_nested(clubs_epl, max_workers, "Premier League")
players_ligue1     = extract_all_player_urls_for_league_parallel_nested(clubs_ligue1, max_workers, "Ligue 1")

# Print contoh output nested
print("\n[Contoh: Klub 1 La Liga]")
for url in players_laliga[0]:
    print(url)
print(f"\n[REKAP JUMLAH PEMAIN PER LIGA]:")
print(f"La Liga     : {sum(len(x) for x in players_laliga)} pemain")
print(f"Serie A     : {sum(len(x) for x in players_seriea)} pemain")
print(f"Bundesliga  : {sum(len(x) for x in players_bundesliga)} pemain")
print(f"Premier Lg  : {sum(len(x) for x in players_epl)} pemain")
print(f"Ligue 1     : {sum(len(x) for x in players_ligue1)} pemain")


[LEAGUE INFO] La Liga: 20 klub | Paralel: 40
[INFO] Klub: https://www.transfermarkt.com/real-madrid/startseite/verein/418/saison_id/2024
      Proxy: 156.228.104.95:3129
      GET: https://www.transfermarkt.com/real-madrid/kader/verein/418/saison_id/2024
[INFO] Klub: https://www.transfermarkt.com/fc-barcelona/startseite/verein/131/saison_id/2024
      Proxy: 156.248.85.158:3129
      GET: https://www.transfermarkt.com/fc-barcelona/kader/verein/131/saison_id/2024
[INFO] Klub: https://www.transfermarkt.com/atletico-madrid/startseite/verein/13/saison_id/2024
      Proxy: 156.228.92.28:3129
      GET: https://www.transfermarkt.com/atletico-madrid/kader/verein/13/saison_id/2024
[INFO] Klub: https://www.transfermarkt.com/athletic-bilbao/startseite/verein/621/saison_id/2024
      Proxy: 156.240.99.247:3129
      GET: https://www.transfermarkt.com/athletic-bilbao/kader/verein/621/saison_id/2024
[INFO] Klub: https://www.transfermarkt.com/real-sociedad-san-sebastian/startseite/verein/681/saison

In [7]:
def scroll_until_stats_loaded(driver, timeout=20, max_scrolls=10):
    for i in range(max_scrolls):
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        perfbox = soup.select_one("tm-player-performance-table")
        if perfbox:
            total_row = perfbox.select_one("div.grid-row.grid-row--dark .grid__cell--ar")
            if total_row and "Total" in total_row.get_text():
                print(f"[DEBUG] Stats section loaded setelah {i+1}x scroll.")
                return soup
        driver.execute_script("window.scrollBy(0, 800);")
        time.sleep(0.2 + random.uniform(0, 0.1))
    print("[ERROR] Stats musim tidak ditemukan setelah scroll maksimal!")
    return BeautifulSoup(driver.page_source, "html.parser")

In [8]:
def parse_int(val):
    if val is None or val.strip() in ["", "-", "'"]:
        return None
    return int(re.sub(r"[^\d]", "", val))

def normalize_url(url):
    # Buang trailing slash, lowercase, dan parameter GET
    base = url.strip().lower().split("?")[0]
    if base.endswith('/'):
        base = base.rstrip('/')
    return base

def extract_profile_and_season_stats(driver, player_url):
    print(f"[PROFILE] Opening profile URL: {player_url}")
    driver.get(player_url)
    click_accept_continue(driver)
    time.sleep(0.3 + random.random() * 0.2)
    soup = scroll_until_stats_loaded(driver)
    print("[PROFILE] Finished initial scroll/load.")

    # Nama
    name = None
    el = soup.select_one(".data-header__headline-wrapper h1")
    if el:
        name = el.get_text(strip=True)
    else:
        meta = soup.find("meta", property="og:title")
        if meta:
            name = meta["content"].split(" - ")[0].strip()
        else:
            title_tag = soup.find("title")
            if title_tag:
                name = title_tag.get_text(strip=True).split(" - ")[0].strip()
    print(f"[PROFILE] Name: {name}")

    # Klub
    club = None
    club_el = soup.select_one(".data-header__club a")
    if not club_el:
        club_el = soup.select_one(".data-header__club .data-header__club-link")
    if club_el:
        club = club_el.get_text(strip=True)
    print(f"[PROFILE] Club: {club}")

    # Data Utama + Caps/Goals Nasional
    birth_date, age, height, citizenship, position, agent = None, None, None, None, None, None
    national_caps, national_goals = None, None  # Tambahan untuk caps & goals timnas

    for ul in soup.select("ul.data-header__items"):
        for li in ul.select("li.data-header__label"):
            label = li.get_text(strip=True)
            if "Date of birth" in label:
                span = li.find_next("span", {"itemprop": "birthDate"})
                if span:
                    raw = span.get_text(strip=True)
                    birth_match = re.match(r"([A-Za-z]+ \d{1,2}, \d{4}) \((\d+)\)", raw)
                    if birth_match:
                        birth_date, age = birth_match.group(1), birth_match.group(2)
                    else:
                        birth_date = raw
            elif "Height" in label:
                span = li.find_next("span", {"itemprop": "height"})
                if span:
                    height = span.get_text(strip=True)
            elif "Citizenship" in label:
                span = li.find_next("span", {"itemprop": "nationality"})
                if span:
                    citizenship = span.get_text(strip=True).split()[-1]
            elif "Position" in label:
                span = li.find_next("span", class_="data-header__content")
                if span:
                    position = span.get_text(strip=True)
            elif "Agent" in label:
                span = li.find_next("span", class_="data-header__content")
                if span:
                    agent = span.get_text(strip=True)
            elif "Caps/Goals" in label:
                print(f"Ada caps {name}")

                # Ambil caps dan goals (biasanya dalam dua <a>)
                caps_goals_li = None
                for li in soup.find_all('li', class_='data-header__label'):
                    if "Caps/Goals:" in li.get_text():
                        caps_goals_li = li
                        break

                if caps_goals_li:
                    # Dari elemen <li> yang ditemukan, cari semua tag <a> dengan kelas yang sesuai
                    a_tags = caps_goals_li.select("a.data-header__content--highlight")
                    if len(a_tags) >= 2:
                        national_caps = a_tags[0].get_text(strip=True)
                        national_goals = a_tags[1].get_text(strip=True)
                        print(f"Caps Nasional: {national_caps}")
                        print(f"Gol Nasional: {national_goals}")
                    else:
                        print("Tidak dapat menemukan caps atau gol yang cukup.")
                else:
                    print("Elemen 'Caps/Goals:' tidak ditemukan.")
    print(f"[PROFILE] Birth Date: {birth_date} | Age: {age} | Height: {height} | Citizenship: {citizenship} | Position: {position} | Agent: {agent}")
    print(f"[PROFILE] National Caps: {national_caps} | National Goals: {national_goals}")

    # Market Value
    market_value = None
    mv = soup.select_one(".data-header__market-value-wrapper")
    if mv:
        val = mv.select_one("span.waehrung")
        if val:
            market_value = mv.get_text(strip=True).replace("Last update:", "").split("\n")[0]
        else:
            market_value = mv.get_text(strip=True)
    print(f"[PROFILE] Market Value: {market_value}")

    # Statistik Musim Terakhir
    stats = {"appearances": None, "goals": None, "assists": None, "minutes": None}
    perfbox = soup.select_one("tm-player-performance-table")
    if not perfbox:
        print("[PROFILE][DEBUG] Tidak menemukan tm-player-performance-table pada profil ini.")
    else:
        grid_rows = perfbox.select("div.grid-row")
        found_total = False
        for row in grid_rows:
            cell_ar = row.select_one(".grid__cell--ar")
            if cell_ar and "Total" in cell_ar.get_text():
                cells = row.select("div.grid__cell--center")
                if position and "keeper" in position.lower():
                    stats["appearances"] = cells[0].get_text(strip=True) if len(cells) > 0 else None
                    stats["goals"] = None
                    stats["assists"] = None
                    stats["minutes"] = cells[3].get_text(strip=True) if len(cells) > 3 else None
                else:
                    stats["appearances"] = cells[0].get_text(strip=True) if len(cells) > 0 else None
                    stats["goals"] = cells[1].get_text(strip=True) if len(cells) > 1 else None
                    stats["assists"] = cells[2].get_text(strip=True) if len(cells) > 2 else None
                    stats["minutes"] = cells[4].get_text(strip=True) if len(cells) > 4 else None
                found_total = True
                break
        if not found_total:
            print("[PROFILE][DEBUG] Tidak ada baris 'Total' pada tm-player-performance-table.")

    print(f"[PROFILE] Season stats: {stats}")

    return {
        "name": name,
        "birth_date": birth_date,
        "age": age,
        "height": height,
        "citizenship": citizenship,
        "position": position,
        "agent": agent,
        "club": club,
        "market_value": market_value,
        "national_caps": national_caps,        # Tambahan output caps
        "national_goals": national_goals,      # Tambahan output goals
        "season_stats": stats
    }

def extract_career_stats(driver, player_url, position):
    career_url = player_url.replace("/profil/", "/leistungsdaten/").split("?")[0]
    if not career_url.endswith("/"):
        career_url += "/"
    career_url += "plus/0?saison=ges"
    print(f"[CAREER] Opening career stats URL: {career_url}")
    driver.get(career_url)
    click_accept_continue(driver)
    max_scrolls = 4
    for scroll_num in range(1, max_scrolls + 1):
        print(f"[CAREER] Scroll ke-{scroll_num} untuk career stats...")
        soup = BeautifulSoup(driver.page_source, "html.parser")
        table = soup.select_one("table.items")
        if not table:
            print(f"[CAREER][DEBUG] table.items belum ditemukan di scroll ke-{scroll_num}")
            driver.execute_script("window.scrollBy(0, 900);")
            time.sleep(0.2 + 0.3 * random.random())
            continue
        headers = [th.get_text(strip=True).lower() for th in table.select("thead th")]
        print(f"[CAREER][DEBUG] Career table headers: {headers}")
        tfoot_tr = soup.select_one("table.items tfoot tr")
        if tfoot_tr and "Total" in tfoot_tr.get_text():
            tds = tfoot_tr.find_all("td")
            print(f"[CAREER][DEBUG] Career stats row: {[td.get_text(strip=True) for td in tds]}")
            if len(tds) < len(headers):
                print("[CAREER][ERROR] Jumlah kolom tfoot tidak sesuai dengan header!")
                return None
            if position and "keeper" in position.lower():
                return {
                    "career_appearances": parse_int(tds[2].get_text()),
                    "career_goals": None,
                    "career_assists": None,
                    "career_minutes": parse_int(tds[-1].get_text())
                }
            else:
                return {
                    "career_appearances": parse_int(tds[2].get_text()),
                    "career_goals": parse_int(tds[3].get_text()) if parse_int(tds[3].get_text()) is not None else None,
                    "career_assists": parse_int(tds[4].get_text()) if parse_int(tds[4].get_text()) is not None else None,
                    "career_minutes": parse_int(tds[-1].get_text())
                }
        else:
            print(f"[CAREER][DEBUG] tfoot/tr 'Total' belum ketemu di scroll ke-{scroll_num}")
        driver.execute_script("window.scrollBy(0, 900);")
        time.sleep(0.5 + 0.3 * random.random())
    print("[CAREER][ERROR] Stats karir tidak ditemukan setelah scroll maksimal!")
    return {
        "career_appearances": None,
        "career_goals": None,
        "career_assists": None,
        "career_minutes": None
    }

def extract_player_profile_selenium(player_url, timeout=15):
    from selenium import webdriver

    results = {}

    def run_profile(results, player_url):
        driver1 = start_driver()
        try:
            data = extract_profile_and_season_stats(driver1, player_url)
            results["profile"] = data
        finally:
            driver1.quit()

    def run_career(results, player_url):
        driver2 = start_driver()
        try:
            pos = None
            for _ in range(100):
                if "profile" in results and results["profile"] is not None:
                    pos = results["profile"].get("position", "")
                    break
                time.sleep(0.01)
            career = extract_career_stats(driver2, player_url, pos)
            results["career_stats"] = career
        finally:
            driver2.quit()

    with ThreadPoolExecutor(max_workers=2) as executor:
        executor.submit(run_profile, results, player_url)
        executor.submit(run_career, results, player_url)
        # Tunggu hingga kedua task selesai
        while "profile" not in results or "career_stats" not in results:
            time.sleep(0.1)

    profile_data = results["profile"]
    career_data = results["career_stats"]
    if profile_data is None:
        print("[ERROR] Gagal ambil profil pemain.")
        return None
    profile_data["career_stats"] = career_data
    profile_data["profile_url"] = player_url

    print("[DEBUG] === ALL DATA COLLECTED ===")
    for k, v in profile_data.items():
        print(f"    {k}: {v}")
    return profile_data

In [9]:
import csv
import threading
from concurrent.futures import ThreadPoolExecutor, as_completed
import os

write_lock = threading.Lock()

def flatten_player_data(player_data):
    flat = {}
    for k, v in player_data.items():
        if isinstance(v, dict):
            for subk, subv in v.items():
                flat[f"{k}_{subk}"] = subv
        else:
            flat[k] = v
    return flat

def normalize_url(url):
    base = url.strip().lower().split("?")[0]
    if base.endswith('/'):
        base = base.rstrip('/')
    return base

def load_existing_urls(csv_path):
    existing_urls = set()
    try:
        with open(csv_path, "r", encoding="utf-8") as f:
            reader = csv.DictReader(f)
            for row in reader:
                if "profile_url" in row:
                    norm = normalize_url(row["profile_url"])
                    existing_urls.add(norm)
    except FileNotFoundError:
        pass
    return existing_urls

def scrape_and_append(players_liga, output_csv=None, max_workers=5):
    # Cek apakah file sudah ada (jangan tulis header ulang)
    header_written = os.path.exists(output_csv) and os.path.getsize(output_csv) > 0

    existing_urls = load_existing_urls(output_csv)
    print(f"[INFO] Found {len(existing_urls)} existing player URLs in {output_csv}, skipping duplicates.")

    def worker(url):
        norm_url = normalize_url(url)
        with write_lock:
            if norm_url in existing_urls:
                print(f"[SKIP] URL already scraped: {url}")
                return False

        data = extract_player_profile_selenium(url)
        if data:
            flat = flatten_player_data(data)
            with write_lock:
                # Write header only once per file (first scrape OR new file)
                nonlocal header_written
                if not header_written:
                    with open(output_csv, "w", newline="", encoding="utf-8") as f:
                        writer = csv.DictWriter(f, fieldnames=sorted(flat.keys()))
                        writer.writeheader()
                    header_written = True

                # Append data
                with open(output_csv, "a", newline="", encoding="utf-8") as f:
                    writer = csv.DictWriter(f, fieldnames=sorted(flat.keys()))
                    writer.writerow(flat)
                existing_urls.add(norm_url)  # Update set supaya real-time skip
            return True
        return False

    all_player_urls = [url for club_urls in players_liga for url in club_urls]
    total = len(all_player_urls)

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(worker, url): url for url in all_player_urls}
        for i, future in enumerate(as_completed(futures), 1):
            url = futures[future]
            try:
                success = future.result()
                if success:
                    print(f"[{i}/{total}] Scraped and saved: {url}")
                else:
                    print(f"[{i}/{total}] Skipped or failed: {url}")
            except Exception as e:
                print(f"[ERROR] {url}: {e}")


In [10]:
import os,csv
all = [
    (players_epl, "..//data//players_epl.csv"),
    (players_laliga, "..//data//players_laliga.csv"),
    (players_bundesliga, "..//data//players_bundes.csv"),
    (players_seriea, "..//data//players_seriea.csv"),
    (players_ligue1, "..//data//playersligue1.csv")
]

for pemain, data in all:
    scrape_and_append(pemain, output_csv=data, max_workers=5)

[INFO] Found 574 existing player URLs in ..//data//players_epl.csv, skipping duplicates.
[SKIP] URL already scraped: https://www.transfermarkt.com/ederson/profil/spieler/238223
[SKIP] URL already scraped: https://www.transfermarkt.com/stefan-ortega/profil/spieler/85941
[SKIP] URL already scraped: https://www.transfermarkt.com/marcus-bettinelli/profil/spieler/116648
[SKIP] URL already scraped: https://www.transfermarkt.com/scott-carson/profil/spieler/14555
[SKIP] URL already scraped: https://www.transfermarkt.com/ruben-dias/profil/spieler/258004
[SKIP] URL already scraped: https://www.transfermarkt.com/abdukodir-khusanov/profil/spieler/763079
[SKIP] URL already scraped: https://www.transfermarkt.com/vitor-reis/profil/spieler/1005575
[SKIP] URL already scraped: https://www.transfermarkt.com/manuel-akanji/profil/spieler/284730
[SKIP] URL already scraped: https://www.transfermarkt.com/john-stones/profil/spieler/186590
[SKIP] URL already scraped: https://www.transfermarkt.com/nathan-ake/pro